# Single label Classifier - AF Detection

Using the dataset provided by the 2020 Physionet Challenge we've developed an Atrial Fibrilation Detector trained to identify AF diagnosed patients from a dataset containing patients with different pathologies like: PAC, RBBB, I-AVB, PVC, LBBB, STD, STE and healthy individuals.

Although data from 12-lead ECG was provided, for this first analysis we've only used the lead 2 data and we've processed the signals in order to create a dataframe consisting of features we believe will help us classify.


In [1]:
import sys
import os
import subprocess
from urllib.parse import urlparse

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from IPython.display import display

from scipy.cluster import hierarchy as hc
from sklearn import metrics
from sklearn.metrics import f1_score, fbeta_score, make_scorer, confusion_matrix, roc_auc_score, recall_score, precision_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, label_binarize
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import mlflow
import mlflow.sklearn

%matplotlib inline
import plotly.express as px
import seaborn as sns

In [2]:
experiment_id = mlflow.set_experiment('features_definitivas')

In [3]:
subprocess.run('rm -rf mlruns/.trash/*', shell=True, check=True)

CompletedProcess(args='rm -rf mlruns/.trash/*', returncode=0)

In [4]:
lead = 'HRV-lead2'
df_raw = pd.read_feather('datasets/phys-raw-lead2-HRV-eda')
df_raw.columns

Index(['HRV_RMSSD', 'HRV_MeanNN', 'HRV_SDNN', 'HRV_SDSD', 'HRV_pNN50',
       'HRV_pNN20', 'HRV_HTI', 'var_P_time', 'var_P_peaks', 'age', 'label'],
      dtype='object')

### Data checks

In [5]:
labels = pd.get_dummies(df_raw['label'])
labels.describe()

,AF,Non-AF
count,6201.000000,6201.000000
mean,0.168037,0.831963
std,0.373930,0.373930
min,0.000000,0.000000
25%,0.000000,1.000000
50%,0.000000,1.000000
75%,0.000000,1.000000
max,1.000000,1.000000


## Split between Train and Validation Datasets

### Training (60%) - Validation (20%) - Evaluation (20%)

In [6]:
y = df_raw['label'].copy()
X = df_raw.drop('label', axis=1)
X_train, X_test, ytrain, ytest = train_test_split(X, y, test_size=0.4, random_state=42)

X_val, X_eval, yval, yeval = train_test_split(X_test, ytest, test_size=0.5, random_state=42)

ytrain[:5]

2770    Non-AF
3790    Non-AF
2277        AF
1401    Non-AF
6179    Non-AF
Name: label, dtype: object

### Label encoding
'AF' as 1 and 'Non-AF' as 0

In [7]:
y_test = label_binarize(ytest, classes=['Non-AF', 'AF'])
y_val = label_binarize(yval, classes=['Non-AF', 'AF'])
y_train = label_binarize(ytrain, classes=['Non-AF', 'AF'])
y_train[:5]

array([[0],
       [0],
       [1],
       [0],
       [0]])

## Evaluation Metrics

This are the evaluation metrics we are actually interested in.

In [8]:
res = pd.DataFrame({'model':[], 'auc train (cv)':[], 'auc val':[], 'f2 val':[], 'sensitivity val':[], 'precision val':[]})

## Model Experiments

In [9]:
models = {
        'LDA': LinearDiscriminantAnalysis(),
        'Perceptron': MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(3, 3), random_state=42, max_iter=100000),
        'Logistic Regression': LogisticRegression(max_iter = 2000, n_jobs = 4, random_state=42),
        'Random Forest': RandomForestClassifier(n_jobs = 4, random_state=42, class_weight={1: 10}),
        'SVC': SVC(random_state=42, class_weight={1: 10}),
        'KNNeighbors': KNeighborsClassifier(n_jobs = 4)
}

#scorer = make_scorer(f1_score, pos_label='AF')

In [ ]:
for name, model in models.items():
    
    # Model training with cross validation
    scores = cross_val_score(model, X_train, y_train, cv=10, scoring='roc_auc', n_jobs=4)
    model.fit(X_train, y_train.ravel())
    
    f2 = fbeta_score(y_val, model.predict(X_val), beta=2)
    roc = roc_auc_score(y_val, model.predict(X_val))
    recall = recall_score(y_val, model.predict(X_val))
    precision = precision_score(y_val, model.predict(X_val))
    
    # Save the results to a dataframe
    res = res.append({'model': f"{name}", 'auc train (cv)': scores.mean(), 'auc val': roc, 'f2 val': f2, 'sensitivity val': recall, 'precision val': precision}, ignore_index=True)
    
    # Experiment Tracking
    with mlflow.start_run(experiment_id=experiment_id):
        mlflow.log_param("model", f"{name}")
        mlflow.log_param("features", "None")
        mlflow.log_param("lead", lead)
        mlflow.log_param("columns", list(X.columns))
        mlflow.log_param("number columns", len(list(X_train.columns)))
        mlflow.log_metric("auc", scores.mean())
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry
        if tracking_url_type_store != "file":

            mlflow.sklearn.log_model(model, "model", registered_model_name="AF_Classifier")
        
        else:
            
            mlflow.sklearn.log_model(model, "model")

In [ ]:
res.sort_values('auc train (cv)', ascending=False)

### Let's try some Scalers

In [ ]:
scal = StandardScaler()
df_normal = scal.fit_transform(df_raw.drop('label', axis=1))
cols = df_raw.columns
cols = cols.drop('label')
df_normal = pd.DataFrame(df_normal, columns=cols)

plt.figure(figsize=(16,6))
sns.boxplot(data=df_normal)
plt.tight_layout()
plt.show()

In [ ]:
scal = StandardScaler()
xtrain_scal = scal.fit_transform(X_train)
xval_scal = scal.transform(X_val)

In [ ]:
for name, model in models.items():
    scores = cross_val_score(model, xtrain_scal, y_train, cv=10, scoring='roc_auc', n_jobs=4)
    model.fit(xtrain_scal, y_train.ravel())
    
    f2 = fbeta_score(y_val, model.predict(xval_scal), beta=2)
    roc = roc_auc_score(y_val, model.predict(xval_scal))
    recall = recall_score(y_val, model.predict(xval_scal))
    precision = precision_score(y_val, model.predict(xval_scal))
    
    res = res.append({'model': f"{name} Scal", 'auc train (cv)': scores.mean(), 'auc val': roc, 'f2 val': f2, 'sensitivity val': recall, 'precision val': precision}, ignore_index=True)
    
    # Experiment Tracking
    with mlflow.start_run(experiment_id=experiment_id):
        mlflow.log_param("model", f"{name}")
        mlflow.log_param("features", "Standard Scaler")
        mlflow.log_param("lead", lead)
        mlflow.log_param("columns", list(X.columns))
        mlflow.log_param("number columns", len(list(X_train.columns)))
        mlflow.log_metric("auc", scores.mean())
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        if tracking_url_type_store != "file":

            mlflow.sklearn.log_model(model, "model", registered_model_name="AF_Classifier")
        
        else:
            
            mlflow.sklearn.log_model(model, "model")

In [ ]:
res.sort_values('auc train (cv)', ascending=False)

### Principal Component Analysis

In [ ]:
n_comps = 5  # +90% explained variance

pca = PCA(n_components = n_comps)

train_pca = pca.fit_transform(xtrain_scal)

val_pca = pca.transform(xval_scal)

In [ ]:
xpca = pd.DataFrame(train_pca)

sns.set_context("talk", font_scale=0.7)
plt.figure(figsize=(15,6))
plt.scatter(xpca.loc[(y_train == 1).ravel(),0],xpca.loc[(y_train == 1).ravel(),1], alpha = 0.3, label = 'AF')
plt.scatter(xpca.loc[(y_train == 0).ravel(),0],xpca.loc[(y_train == 0).ravel(),1], alpha = 0.3, label = 'Non-AF')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Principal Component Analysis before feature selection')
plt.legend(loc='upper right')
plt.show()

In [ ]:
for name, model in models.items():
    scores = cross_val_score(model, train_pca, y_train, cv=10, scoring='roc_auc', n_jobs=4)
    
    model.fit(train_pca, y_train.ravel())
    
    f2 = fbeta_score(y_val, model.predict(val_pca), beta=2)
    roc = roc_auc_score(y_val, model.predict(val_pca))
    recall = recall_score(y_val, model.predict(val_pca))
    precision = precision_score(y_val, model.predict(val_pca))
    
    res = res.append({'model': f"{name} PCA", 'auc train (cv)': scores.mean(), 'auc val': roc, 'f2 val': f2, 'sensitivity val': recall, 'precision val': precision}, ignore_index=True)
    
    # Experiment Tracking
    with mlflow.start_run(experiment_id=experiment_id):
        mlflow.log_param("model", f"{name}")
        mlflow.log_param("features", "PCA")
        mlflow.log_param("lead", lead)
        mlflow.log_param("columns", list(X.columns))
        mlflow.log_param("number columns", len(list(X_train.columns)))
        mlflow.log_metric("auc", scores.mean())
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        if tracking_url_type_store != "file":

            mlflow.sklearn.log_model(model, "model", registered_model_name="AF_Classifier")
        
        else:
            
            mlflow.sklearn.log_model(model, "model")

In [ ]:
res.sort_values('auc val', ascending=False)